In [1]:
!pip install transformers sentence_transformers accelerate bitsandbytes chromadb evaluate rouge_score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import files

# Загрузка файла через интерфейс Colab
uploaded = files.upload()

# После загрузки читаем файл (в примере используется текстовый файл)
for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as file:
        document_text = file.read()

# Функция для разбиения текста на чанки
def split_into_chunks(text, chunk_size=200):
    words = text.split()
    return [' '.join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# Разбиваем документ на чанки (по 200 слов)
chunks = split_into_chunks(document_text, chunk_size=200)
print(f"Количество чанков: {len(chunks)}")

Saving HR-бот.txt to HR-бот.txt
Количество чанков: 25


In [3]:
import chromadb
import evaluate
from sentence_transformers import SentenceTransformer

# Инициализация модели SentenceTransformer
model = SentenceTransformer("deepvk/USER-bge-m3")

# Создание клиента ChromaDB
client = chromadb.Client()

rouge = evaluate.load("rouge")

# Загрузка метрики METEOR
meteor = evaluate.load("meteor")


# Создаем коллекцию в ChromaDB без указания функции для эмбеддингов (мы сами будем генерировать их)
collection = client.create_collection(name="rag_texts")

# Генерируем эмбеддинги для чанков с использованием модели SentenceTransformer
embeddings = model.encode(chunks)  # chunks - это твои текстовые данные, которые нужно закодировать

# Добавляем чанки в коллекцию с эмбеддингами
chunk_ids = [str(i) for i in range(len(chunks))]
collection.add(documents=chunks, embeddings=embeddings, ids=chunk_ids)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.34k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.33M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "facebook/xglm-2.9B"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/5.88G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
def get_prompt(user_query, context):
  return f"""Ты специалист тех поддержки, которые отвечает на вопросы пользователей по технической документации.
Тебе нужно отвечать на вопросы строго по документации, не придумывая ничего самому.
Отвечай на вопросы развернуто.
Не генерируй примеры кода, которые демонстрируют, как можно обрабатывать вопросы и ответы.
Отвечай на вопросы в деловом тоне.
Над ответом на вопрос думаю пошагово.
В ответах ни как не упоминай, что ты отвечаешь на вопрос по документации или какому-либо контексту.
В ответе не упоминай вопрос пользователя.
Не указывай никаких ссылок, если они не были заранее даны в документации.
Если ответа нет в документации, тогда пиши я не знаю.
content : отвечай на вопрос по этому контенксту: {context}.

user : вопрос пользователя звучит так: {user_query}"""

In [16]:
generation_config = GenerationConfig(
    bos_token_id=128000,         # начальный токен, c которого начинается генерация
    eos_token_id=128009,         # токен остановки генерации
    pad_token_id=128000,         # id паддинга(одинаковая длина последовательностей)
    do_sample=False,             # Используем семплирование
    max_new_tokens=512,          # Ограничиваем количество новых токенов
    repetition_penalty=1.5,      # Штраф за повторения
    temperature=0.2,             # Степень случайности
    top_k=30,                    # Сужаем выбор токенов до k
    top_p=0.5                   # Ограничиваем вероятность выбора
)
print(generation_config)

def similarity_search(collection, user_query, n_results=4):
    query_embedding = SentenceTransformer("deepvk/USER-bge-m3").encode([user_query])
    results = collection.query(query_embeddings=query_embedding, n_results=n_results)
    return results['documents'][0]

def rag_with_chunks(query):
    # Ищем релевантные чанки
    retrieved_chunk = similarity_search(collection, query)

    # Создаем промпт с найденным контекстом
    prompt = get_prompt(query, retrieved_chunk)

    # Генерация ответа с помощью модели
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]):]  # Убираем токены промпта
    output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

    return output

references = """В корпоративном ТГ-канале @USSC_ltd (https://t.me/+Ccat7WcyfM9iN2Yy) мы публикуем самые срочные и важные новости: письма от директоров, информацию о ДМС, анонсы мероприятий и т.д.
"""


# Пример запроса
query = "Как перейти в корпоративный ТГ- канал?"
answer = rag_with_chunks(query)


rouge_result = rouge.compute(predictions=[answer], references=[references])
print("ROUGE:", rouge_result)

# Подсчет METEOR
meteor_result = meteor.compute(predictions=[answer], references=[references])
print("METEOR:", meteor_result)

print(f"Ответ: {answer}")


GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "max_new_tokens": 512,
  "pad_token_id": 128000,
  "repetition_penalty": 1.5,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.5
}

ROUGE: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
METEOR: {'meteor': 0.032327586206896554}
Ответ: © 2017 ООО "Агентский дом" _ Политика конфиденциальности персональны данных Пользователей сайта www3dnews24.com / Сайт создан студиЕЙ 3DVISION® г Екатеринбург • EMAIL info[at ]uasb2k2.org• Телефон + 7 (343) 222-22-20© 2018 Студия дизайна uaSBSTM All Rights Reserved Создание сайтов — компания “ТридэВизия” ® 2016–2018 гг Карта проезда Контакты Реклама Соглашение Конфиденциальность Условия использования материалов Дизайн от студии SRS Design Studio Разработка логотиповой концепциции web design by studio srsdesignstudio создание интернет магазина megagroup ru разработка мобильности дизайн упаковкасопровождением клиентов продвижение брендирования рекламными кампаниями seo